In [1]:
%pip install pdfplumber pytesseract pdf2image

  Using cached pdfplumber-0.11.9-py3-none-any.whl.metadata (43 kB)
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pdfminer_six-20251230-py3-none-any.whl.metadata (4.3 kB)
  Using cached pypdfium2-5.3.0-py3-none-win_amd64.whl.metadata (67 kB)
Using cached pdfplumber-0.11.9-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20251230-py3-none-any.whl (6.6 MB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Using cached pypdfium2-5.3.0-py3-none-win_amd64.whl (3.1 MB)

   ---------------------------------------- 0/5 [pytesseract]
   -------- ------------------------------- 1/5 [pypdfium2]
   -------- ------------------------------- 1/5 [pypdfium2]
   -------- ------------------------------- 1/5 [pypdfium2]
   -------- ------------------------------- 1/5 [pypdfium2]
   -------- ------------------------------- 1/5 [pypdfium2]
  

In [2]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Try direct text extraction
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"Direct text extraction failed: {e}")

    # Fallback to OCR for image-based PDFs
    print("Falling back to OCR for image-based PDF.")
    try:
        images = convert_from_path(pdf_path)
        for image in images:
            page_text = pytesseract.image_to_string(image)
            text += page_text + "\n"
    except Exception as e:
        print(f"OCR failed: {e}")

    return text.strip()

In [4]:
pdf_path = "Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)

print("\nExtracted Text from PDF:")
print(resume_text)

Direct text extraction failed: [Errno 2] No such file or directory: 'Resume.pdf'
Falling back to OCR for image-based PDF.
OCR failed: Unable to get page count. Is poppler installed and in PATH?

Extracted Text from PDF:



In [6]:
%pip install langchain-groq python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install langchain-groq


Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_groq import ChatGroq

# Define API key directly in the notebook cell (Jupyter use-case)
GROQ_API_KEY = "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"

# Initialize ChatGroq model
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.1-8b-instant",
    temperature=0
)

# Invoke the model
response = llm.invoke("explain langchain in one line?")

# Print only the model output text
print(response.content)


c:\Users\mansi\OneDrive\Desktop\JD Resume Anlyzer system\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LangChain is an open-source framework that enables developers to build, train, and deploy large language models (LLMs) and other AI models, allowing for the creation of custom conversational AI applications.


In [9]:
from langchain_groq import ChatGroq

# Define API key directly (Jupyter use-case)
GROQ_API_KEY = "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"

def analyze_resume(resume_text, job_description=None):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    # Initialize Groq LLM
    model = ChatGroq(
        api_key=GROQ_API_KEY,
        model="llama-3.1-8b-instant",
        temperature=0
    )
    
    base_prompt = f"""
    You are an experienced HR with Technical Experience in the field of any one job role from Data Science, Data Analyst, DevOPS, Machine Learning Engineer, Prompt Engineer, AI Engineer, Full Stack Web Development, Big Data Engineering, Marketing Analyst, Human Resource Manager, Software Developer your task is to review the provided resume.
    Please share your professional evaluation on whether the candidate's profile aligns with the role.ALso mention Skills he already have and siggest some skills to imorve his resume , alos suggest some course he might take to improve the skills.Highlight the strengths and weaknesses.

    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements.
        """

    # Invoke Groq model
    response = model.invoke(base_prompt)

    analysis = response.content.strip()
    return analysis


In [10]:
print(analyze_resume(resume_text))

{'error': 'Resume text is required for analysis.'}


In [ ]:
def extract_candidate_profile(resume_text):
    model = ChatGroq(
        api_key=GROQ_API_KEY,
        model="llama-3.1-8b-instant",
        temperature=0
    )

    extraction_prompt = f"""
You are an AI system specialized in extracting structured candidate profiles from resumes.

TASK:
Extract structured, normalized candidate information from the resume text.

EXTRACTION RULES:
- Extract ONLY information explicitly present in the resume
- Do NOT infer or assume missing details
- Normalize skill names (e.g., Python, SQL, AWS)
- If a section is missing, return an empty list or empty string
- Return VALID JSON ONLY (no explanation, no markdown)

OUTPUT FORMAT (STRICT JSON):

{{
  "candidate_profile": {{
    "summary": "",
    "skills": {{
      "technical": [],
      "tools": [],
      "frameworks": [],
      "databases": [],
      "cloud": []
    }},
    "experience": [
      {{
        "role": "",
        "company": "",
        "duration": "",
        "responsibilities": [],
        "technologies": []
      }}
    ],
    "projects": [
      {{
        "name": "",
        "description": "",
        "technologies": [],
        "outcome": ""
      }}
    ],
    "education": [
      {{
        "degree": "",
        "institution": "",
        "year": ""
      }}
    ]
  }}
}}

-----------------------
RESUME TEXT:
{resume_text}
"""

    response = model.invoke(extraction_prompt)
    return response.content.strip()
